# It Takes a Village: Experimental Evidence on the Impact of Community Currencies in Kenya

This document outlines the program design and implementation of a Randomized Control Trial (RCT) aimed at understanding the impact of Community Currencies (CCs) as an effective form of Unconditional Cash Transfer (UCT). 

### Potential hypotheses:

**Hypothesis 1**: Community currency airdrops (a form of digital unconditional cash transfer) have no effect on the diversity of individual economic engagement, as measured by number of trade partners, frequency of trade, and categories of trade. This hypothesis assumes recipients of airdrops do not change their trading behaviour or economic reach within a local network beyond the increase in amount equal to the airdrop size.
 
**Hypothesis 2**: Community currency airdrops have equal effects for both men and women. This hypothesis assumes that the impact of unconditional cash transfers is not determined by gender.
 
**Hypothesis 3**: (?)

### Program design and execution

**Location**: Nairobi, Kenya

**Target population**: 666,8 users of the Sarafu Network, a CC program that has been operating throughout Kenya for over a decade.

Intervention: All eligible individuals receive a total transfer of 400 Sarafu (approximately $4.00), split into three instalments over the course of three months:

  Week 1: 400 Sarafu (USD 4.00)
  
  Week 2: 400 Sarafu (USD 4.00)
  
  Week 3: 400 Sarafu (USD.00)

The instalments are sized to match the standard amount of Sarafu users receive when joining the network for the first time. Transfers are sent via the Sarafu blockchain and this is visible to users as a top-up to their wallet, accessible via USSD commands on their mobile phones.


### Study design and data collection

Given the unique nature of this remote, low-cost RCT, constraints are placed on the number of observable variables. Outcomes measured are all trade-specific and sourced from the Sarafu blockchain, including:


* Trade amount
* Trade frequency
* Number of trade partners
* Category of items traded
* Gender of trade partners

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import stats
import seaborn as sns
import datetime
from datetime import datetime
import random
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

import statistics 
from statistics import mode 
from collections import Counter

In [3]:
txnData = pd.read_csv('txnData_Sep_2020.csv', header=0, index_col=None)
txnData['timeset'] = pd.to_datetime(txnData['timeset']).dt.floor('d')

print('{} transactions'.format(len(txnData)))
txn_locations = list(enumerate(sorted(txnData['s_location_path'].unique())))
print('Total number of unique txn locations:', len(txn_locations))
txn_users = list(enumerate(sorted(txnData['source'].unique())))
print('Total number of unique users:', len(txn_users))

txnData.transfer_use = txnData.transfer_use.astype(str).str.lower()
txnData.transfer_use = txnData.s_business_type.astype(str).str.lower()
txnData.transfer_use = txnData.t_business_type.astype(str).str.lower()
txnData.loc[(txnData.transfer_use == 'chama'),'transfer_use'] = 'savings group'
txnData.loc[(txnData.transfer_use == 'group'),'transfer_use'] = 'savings group'
txnData.loc[(txnData.transfer_use == 'test'),'transfer_use'] = 'system'
txnData.loc[(txnData.transfer_use == 'staff'),'transfer_use'] = 'system'
txnData.transfer_use = txnData.transfer_use.astype('category').cat.codes

# Where people forget to enter transfer_use, assume it will be for whatever the recipients' business type is
txnData.loc[(txnData.transfer_use == 'nan'),'transfer_use'] = txnData.t_business_type
txnData.loc[(txnData.transfer_use == 'non'),'transfer_use'] = txnData.t_business_type

# Make gender inputs consistent
txnData.s_gender = txnData.s_gender.str.lower()
txnData.t_gender = txnData.s_gender.str.lower()
txnData.s_gender = txnData.s_gender.replace('male ', 'male')
txnData.s_gender = txnData.s_gender.replace('male', 0)
txnData.s_gender = txnData.s_gender.replace('female', 1)
txnData = txnData[(txnData.s_gender != 'other') & (txnData.s_gender != 'unknown gender')]
#txnData.dropna(subset=['s_gender'], inplace=True)

txnData.head()

/Users/rebeccamqamelo/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


418998 transactions
Total number of unique txn locations: 70
Total number of unique users: 33637


/Users/rebeccamqamelo/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,id,timeset,transfer_subtype,transfer_use,source,s_comm_tkn,s_gender,s_location_path,s_location_lat,s_location_lon,s_business_type,target,t_comm_tkn,t_gender,t_location_path,t_location_lat,t_location_lon,t_business_type,tx_token,weight,type,token_name,token_address
0,1,2020-01-25,DISBURSEMENT,9,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,None,NaN,None,None,None,System,0x245fc81fe385450Dc0f4787668e47c903C00b0A1,None,NaN,None,None,None,Savings Group,nan,"18,000.00",directed,Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
1,2,2020-01-25,DISBURSEMENT,2,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,None,NaN,None,None,None,System,0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2,None,NaN,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,Farming/Labour,nan,"9,047.66",directed,Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
2,3,2020-01-25,DISBURSEMENT,2,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,None,NaN,None,None,None,System,0xBAB77A20a757e8438DfaBF01D5F36DD12d862B31,None,NaN,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,Farming/Labour,nan,"25,378.73",directed,Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
3,4,2020-01-25,DISBURSEMENT,6,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,None,NaN,None,None,None,System,0xD95954e3fCd2f09A6Be5931D24f731eFa63BF435,None,NaN,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,Health,nan,"4,495.93",directed,Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
4,5,2020-01-26,DISBURSEMENT,2,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,None,NaN,None,None,None,System,0x4AB73CfaC1732a9DcD74BdB4C9605f21832D7C72,None,NaN,None,None,None,Farming/Labour,nan,400.00,directed,Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4


In [4]:
userData = pd.read_csv('userData_cleaned_beneficiary_only.csv', header=0, index_col=None)
userData.drop(columns=['Unnamed: 0'], inplace=True)

userData.drop(userData[userData['held_roles'] == 'ADMIN'].index, inplace=True)
userData.drop(userData[userData['xDAI_blockchain_address'] == 'None'].index, inplace=True)
userData['start'] = pd.to_datetime(userData['start']).dt.floor('d')
print('{} entries'.format(len(userData)))
print('{} unique users'.format(len(userData.xDAI_blockchain_address.unique())))
user_locations = list(enumerate(sorted(userData['location'].unique()))) 
print('Total number of unique user locations:', len(user_locations))

userData.business_type = userData.business_type.astype(str).str.lower()
userData.loc[(userData.business_type == 'chama'),'business_type'] = 'savings group'
userData.loc[(userData.business_type == 'group'),'business_type'] = 'savings group'
userData.loc[(userData.business_type == 'test'),'business_type'] = 'system'
userData.loc[(userData.business_type == 'staff'),'business_type'] = 'system'
user_types = list(enumerate(sorted(userData['business_type'].unique())))
print('User business types:', user_types)

# Make gender inputs consistent
userData.gender = userData.gender.str.lower()
userData.gender = userData.gender.replace('male ', 'male')
userData.gender = userData.gender.replace('male', 0)
userData.gender = userData.gender.replace('female', 1)
userData = userData[(userData.gender != 'other') & (userData.gender != 'unknown gender')]
userData.dropna(subset=['gender'], inplace=True)

userData.head()

37851 entries
37851 unique users
Total number of unique user locations: 72
User business types: [(0, 'education'), (1, 'environment'), (2, 'farming/labour'), (3, 'food/water'), (4, 'fuel/energy'), (5, 'game'), (6, 'health'), (7, 'm-pesa'), (8, 'none'), (9, 'savings group'), (10, 'shop'), (11, 'system'), (12, 'transport'), (13, 'volunteer')]


,id,start,label,gender,location,lat,lon,held_roles,business_type,bal,xDAI_blockchain_address,ovol_in,ovol_out,otxns_in,otxns_out,ounique_in,ounique_out,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,days_enrolled,days_active,trades_in_daily,trades_out_daily,trade_partners_in_daily,trade_partners_out_daily
0,14899,2020-04-18,14899,1,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,"4,023.83",0x88c23f101c23b687AB601c14f1057dFe22b7785C,"6,352.83",0.00,30,0,7,0,"3,650,672.00","3,653,001.00",1153,966,37,19,151,146,7.90,6.62,0.25,0.13
1,14897,2020-04-18,14897,0,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,0.87,0x5671e92a1Fbe24B5609251f1BfEE7Dd6319BA963,"7,153.87",0.00,32,0,5,0,"2,502,170.00","2,509,323.00",598,544,43,25,151,145,4.12,3.75,0.30,0.17
2,17919,2020-04-29,17919,1,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,600.89,0x632917835d2a6C91beEe2E4Cc2B4Af1E5b3aEc51,"8,710.89",0.00,31,0,6,0,"2,118,907.00","2,127,017.00",546,625,41,30,140,136,4.01,4.60,0.30,0.22
3,15902,2020-04-22,15902,0,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,shop,62.19,0x46b40003f9b3C8899e8C996BF9A174F9C29D3950,"4,922.20",0.00,19,0,4,0,"1,832,325.10","1,837,185.11",347,330,20,21,147,100,3.47,3.30,0.20,0.21
4,15318,2020-04-20,15318,0,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,473.67,0x389b28D1371b83c669e110b605B1fA9afBDf9fc9,"4,598.56",0.00,23,0,3,0,"1,555,594.11","1,559,719.00",311,346,18,19,149,146,2.13,2.37,0.12,0.13


### Selection of treatment and control in Nairobi

### Method

Study sample: The study takes place in Nairobi amongst a population of 6,668 Sarafu users. Individuals are filtered for sample eligibility based on their trade frequency.

Individuals are filtered into four subgroups based on their trade frequency, with groups 2-4 being included in the study and randomized to treatment or control groups, using propensity score matching based on their baseline characteristics. 

The eligible population sample is divided into a male-only subgroup and female-only sub-group, each of which are further separated into treatment and control.

**Unit of randomization**: Community and individual
**Unit of outcome observation**: Individual

In [4]:
userData_N = userData[userData.location.str.contains('Nairobi', regex=False)]
userData_N_sub_locations = list(enumerate(sorted(userData_N['location'].unique()))) 
print('Total number of unique sub-locations in Nairobi:', len(userData_N_sub_locations))
print('Total number of users in Nairobi:', len(userData_N))
userData_N.head()

Total number of unique sub-locations in Nairobi: 27
Total number of users in Nairobi: 6668


,id,start,label,gender,location,lat,lon,held_roles,business_type,bal,xDAI_blockchain_address,ovol_in,ovol_out,otxns_in,otxns_out,ounique_in,ounique_out,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,days_enrolled,days_active,trades_in_daily,trades_out_daily,trade_partners_in_daily,trade_partners_out_daily
0,14899,2020-04-18,14899,1,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,"4,023.83",0x88c23f101c23b687AB601c14f1057dFe22b7785C,"6,352.83",0.00,30,0,7,0,"3,650,672.00","3,653,001.00",1153,966,37,19,151,146,7.90,6.62,0.25,0.13
1,14897,2020-04-18,14897,0,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,0.87,0x5671e92a1Fbe24B5609251f1BfEE7Dd6319BA963,"7,153.87",0.00,32,0,5,0,"2,502,170.00","2,509,323.00",598,544,43,25,151,145,4.12,3.75,0.30,0.17
2,17919,2020-04-29,17919,1,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,600.89,0x632917835d2a6C91beEe2E4Cc2B4Af1E5b3aEc51,"8,710.89",0.00,31,0,6,0,"2,118,907.00","2,127,017.00",546,625,41,30,140,136,4.01,4.60,0.30,0.22
3,15902,2020-04-22,15902,0,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,shop,62.19,0x46b40003f9b3C8899e8C996BF9A174F9C29D3950,"4,922.20",0.00,19,0,4,0,"1,832,325.10","1,837,185.11",347,330,20,21,147,100,3.47,3.30,0.20,0.21
4,15318,2020-04-20,15318,0,"Mukuru Kayaba Village,Plainsview Road,South B,...",36.8445084441166,-1.3112602,BENEFICIARY,farming/labour,473.67,0x389b28D1371b83c669e110b605B1fA9afBDf9fc9,"4,598.56",0.00,23,0,3,0,"1,555,594.11","1,559,719.00",311,346,18,19,149,146,2.13,2.37,0.12,0.13


In [26]:
print('Total nairobi population:', len(userData_N))
sample = userData_N.loc[(userData_N.days_active >= 30) & (userData_N.trades_out_daily > 1) & (userData_N.trades_in_daily > 1)]
print('Sample size in Nairobi =', len(sample))

Total nairobi population: 6668
Sample size in Nairobi = 210


In [27]:
women = sample.loc[(sample.gender == 1)]
print('W:', len(women))
men = sample.loc[(sample.gender == 0)]
print('M:', len(men))

W: 159
M: 51


In [29]:
women_trt = pd.DataFrame(random.sample(women['xDAI_blockchain_address'].tolist(), k=113))
men_trt = pd.DataFrame(random.sample(men['xDAI_blockchain_address'].tolist(), k=46))
trt = pd.concat([women_trt, men_trt])
print(len(trt))
trt.head()

159


,0
0,0x665d425405800C5d0D83d348545a4345f423Bf35
1,0xCede514262a4044Ce4a88cE58Afbf8DA6FFA2a85
2,0x8D3EFEF7096884794bBaFe37f0D371af07a65bfC
3,0xa8284A3580f19e437A0298dE012756Ab2E14Bb67
4,0xCf5A5451a91786123738e0F983b1Cb7202245601


In [17]:
trt.to_csv('Rebecca_treatment_wallets_1.0.csv')

### Testing if this selection still works with new data

In [5]:
userData_new= pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/users_all_pub_20200125-20201109-all_time.csv', header=0, index_col=None)
userData_new.head()

,id,start,label,gender,location,lat,lon,held_roles,business_type,bal,xDAI_blockchain_address,ovol_in,ovol_out,otxns_in,otxns_out,ounique_in,ounique_out,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out,sunique_out_group,sunique_in_at,sunique_out_at,sunique_out_at_group,days_enrolled,days_active,trades_in_daily,trades_out_daily,trade_partners_in_daily,trade_partners_out_daily,gender_ratio_in,gender_ratio_out
0,4,2018-10-21,4,male,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,BENEFICIARY,Farming/Labour,"2,448.89",0xBAB77A20a757e8438DfaBF01D5F36DD12d862B31,"86,049.79","44,075.00",13,2,2,1,"22,744.10","62,270.00",32,32,23,20,0,19,18,0,753,282,0.11,0.11,0.08,0.07,0.00,0.73
1,3,2018-10-21,3,male,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,BENEFICIARY,Farming/Labour,0.66,0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2,"9,103.66",96.00,2,3,2,1,0.00,"9,007.00",0,1,0,1,0,0,1,0,753,269,0.00,0.00,0.00,0.00,0.00,0.00
2,5,2018-10-23,5,male,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,BENEFICIARY,Health,194.53,0xD95954e3fCd2f09A6Be5931D24f731eFa63BF435,"64,694.23",300.00,35,2,2,1,"247,335.30","311,535.00",291,296,177,147,0,173,144,0,751,282,1.03,1.05,0.63,0.52,0.00,0.91
3,2,2018-10-23,2,female,"Kilifi,Kilifi,Coastal Kenya",39.85,-3.6333,TOKEN_AGENT,Savings Group,"1,864.00",0x245fc81fe385450Dc0f4787668e47c903C00b0A1,"6,190,057.95","6,173,193.95",271,33,125,1,0.00,"15,000.00",0,1,0,1,0,0,1,0,751,282,0.00,0.00,0.00,0.00,2.89,0.00
4,16,2018-10-24,16,female,None,None,None,BENEFICIARY,Farming/Labour,315.68,0xfDa3ab16Ad9AE93a09fAE301e08035d1CF970D95,656.68,441.00,2,4,2,1,100.00,0.00,1,0,1,0,0,1,0,0,750,281,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
load_trt = pd.read_csv('Rebecca_treatment_wallets_1.0.csv', header=0, index_col=None)
load_trt.head()

,Unnamed: 0,0
0,0,0xbfA95c2D69B5a3E4b2BDeB473A326e31A893b5B7
1,1,0x89f1cbac4670c8334544C65ABfdf88327c970a61
2,2,0x92BD639eb257C0DC3AC10200fFeEe52ad2803e86
3,3,0x0d82e4Bd9d204C8ebECFA41b67C99a76CcB64C8C
4,4,0x804cD618E1a65f5ba789184c9386f7C0509bF7ae


### Run the same analysis on the new data to check

Running the new trt group on the same filter resulted in only 108 eligible women and 32 eligible men in Nairobi, so below the filter are relaxed a bit to make the study size larger.

In [9]:
userData_N = userData_new[userData_new.location.str.contains('Nairobi', regex=False)]
userData_N_sub_locations = list(enumerate(sorted(userData_N['location'].unique()))) 
print('Total nairobi population:', len(userData_N))
sample = userData_N.loc[(userData_N.days_active >= 30) & (userData_N.trades_out_daily > 1) & (userData_N.trades_in_daily > 1)]
print('Sample size in Nairobi =', len(sample))
women = sample.loc[(sample.gender == 'female')]
print('W:', len(women))
men = sample.loc[(sample.gender == 'male')]
print('M:', len(men))

Total nairobi population: 11446
Sample size in Nairobi = 161
W: 108
M: 32


In [ ]:
women_trt = pd.DataFrame(random.sample(women['xDAI_blockchain_address'].tolist(), k=85))
men_trt = pd.DataFrame(random.sample(men['xDAI_blockchain_address'].tolist(), k=30))
trt_new = pd.concat([women_trt, men_trt])
print(len(trt_new))
trt_new.head()
trt_new.to_csv('Rebecca_treatment_wallets_LATEST.csv')